# Necessary Imports and Settings

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import transformers
import torch
import os
import nltk
import pandas as pd
import torch
import numpy as np
from jinja2 import Template
import xmltodict
import pickle
from collections import defaultdict
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import time
import copy

import sys
sys.path.append('/scratch/users/bozyurt20/hpc_run/utilities')
sys.path.append("/scratch/users/bozyurt20/hpc_run/blobs/")
from util_research import *

#from toy_dataset import contexts

max_len = 512
num_layers = 24
d_model = 4096

tokenizer = AutoTokenizer.from_pretrained("bigscience/T0pp", truncation_side="right", add_prefix_space=True)

In [2]:
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0pp", device_map="balanced", load_in_8bit=True)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /kuacc/users/bozyurt20/.conda/envs/hf/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda110.so...


In [3]:
names_1 = ["John", "Harry", "Andrew", "Lisa", "Mary"]
names_2 = ["Henry", "David", "Sophia", "Olivia", "Emma"]

cities_1 = ["London", "Paris", "Oslo", "Istanbul", "Beijing"]
cities_2 = ["Sydney", "Cairo", "Seoul", "Rome", "Prague"]

class DataSample_TwoSentences():
    def __init__(self, name_1, name_2, city_1, city_2):
        self.name_1 = name_1
        self.name_2 = name_2
        self.city_1 = city_1
        self.city_2 = city_2
        self.prev_context = name_1 + " travelled to " + city_1 + ". " + name_2 + " travelled to " + city_2 + "."
        self.current_context_1 = name_1 + " met Lucas. Lucas was 30 years old. Where is " + name_1 + "?"
        self.current_context_2 = name_2 + " met Lucas. Lucas was 30 years old. Where is " + name_2 + "?"
        
    def add_prev_encoding(self, encoding):
        self.prev_encoding = encoding
    def add_current_encoding_1(self, encoding):
        self.curr_encoding_1 = encoding
    def add_current_encoding_2(self, encoding):
        self.curr_encoding_2 = encoding
        
class TwoSentenceMoveResult():
    def __init__ (self, data_point, prev_context, answer_1, answer_2, prompt_1, prompt_2, model_out_1, model_out_2, out_generated_1, out_generated_2):
        
        self.data_point = data_point
        self.prev_context = prev_context
        self.answer_1 = answer_1
        self.answer_2 = answer_2 
        self.prompt_1 = prompt_1
        self.prompt_2 = prompt_2
        self.model_out_1 = model_out_1
        self.model_out_2 = model_out_2
        self.out_generated_1 = out_generated_1
        self.out_generated_2 = out_generated_2
        
class TwoSentenceMoveResultCase2():
    def __init__ (self, data_point, prev_context, answer_1, answer_2, model_out_1, model_out_2, out_generated_1, out_generated_2):
        
        self.data_point = data_point
        self.prev_context = prev_context
        self.answer_1 = answer_1
        self.answer_2 = answer_2
        self.model_out_1 = model_out_1
        self.model_out_2 = model_out_2
        self.out_generated_1 = out_generated_1
        self.out_generated_2 = out_generated_2

In [13]:
def case1_generation(data_points_two_sentences_case1, ids_to_move, entity_inds):

    results_case1 = []

    for num_data_point, data_point in enumerate(data_points_two_sentences_case1):

        print("NOW PROCESSING:", num_data_point)
        start = time.time()

        name_1 = data_point.name_1
        name_2 = data_point.name_2
        answer_1 = data_point.city_1
        answer_2 = data_point.city_2
        prev_context = data_point.prev_context
        current_context_1 = data_point.current_context_1
        current_context_2 = data_point.current_context_2
        prev_encoding = data_point.prev_encoding

        special_hidden = prev_encoding.special_hidden_states # 24 x (1, T, d)
        len_input_ids = special_hidden[0].shape[1]

        special_reformatted = torch.zeros(num_layers, len_input_ids, d_model) # (24, T, d)
        for i, hidden in enumerate(special_hidden):
            special_reformatted[i:i+1, :, :] = hidden

        entities_hidden_states = special_reformatted[:, ids_to_move[0], :].unsqueeze(0) # 1, 24, d

        for tok_id in ids_to_move[1:]:
            entity_hidden_states = special_reformatted[:, tok_id, :].unsqueeze(0) # 1, 24, d
            entities_hidden_states = torch.cat((entities_hidden_states,
                                                entity_hidden_states), dim=0)

        if ids_to_move == [ 0, 6, 4, 10 ]:
            tokens_to_prepend = [name_1, name_2, answer_1, answer_2]
        elif ids_to_move == [ 4, 10 ]:
            tokens_to_prepend = [answer_1, answer_2]
        elif ids_to_move == [ 0, 6 ]:
            tokens_to_prepend = [name_1, name_2]
        elif ids_to_move == [ 3, 4, 9, 10 ]:
            tokens_to_prepend = ["to", answer_1, "to", answer_2]
        elif ids_to_move == [ 0, 6, 3, 4, 9, 10 ]:
            tokens_to_prepend = [name_1, name_2, "to", answer_1, "to", answer_2]

        prompt_1 = " ".join(tokens_to_prepend) + " " + current_context_1
        input_ids = tokenizer.encode(prompt_1, return_tensors="pt").to(model.encoder.device)
        model_out_1 = model.generate(input_ids=input_ids, 
                                     max_new_tokens=10,
                                     entity_hidden_states=entities_hidden_states,
                                     entity_inds=entity_inds,
                                     return_dict_in_generate=True, 
                                     output_scores=True)

        prompt_2 = " ".join(tokens_to_prepend) + " " + current_context_2
        input_ids = tokenizer.encode(prompt_2, return_tensors="pt").to(model.encoder.device)
        model_out_2 = model.generate(input_ids=input_ids, 
                                     max_new_tokens=10,
                                     entity_hidden_states=entities_hidden_states,
                                     entity_inds=entity_inds,
                                     return_dict_in_generate=True, 
                                     output_scores=True) 


        out_generated_1 = tokenizer.decode(model_out_1.sequences[0], skip_special_tokens=True)
        out_generated_2 = tokenizer.decode(model_out_2.sequences[0], skip_special_tokens=True)

        results_case1.append(TwoSentenceMoveResult(data_point, prev_context, answer_1, answer_2, prompt_1, prompt_2, model_out_1, model_out_2, out_generated_1, out_generated_2))

        print("Time taken:", time.time()-start)
    return results_case1


In [22]:
def case2_generation(data_points_two_sentences_case2, ids_to_move):
    
    results_case2 = []

    for num_data_point, data_point in enumerate(data_points_two_sentences_case2):

        print("NOW PROCESSING:", num_data_point)
        start = time.time()

        answer_1 = data_point.city_1
        answer_2 = data_point.city_2
        prev_context = data_point.prev_context
        entity_embeddings = data_point.prev_encoding
        curr_embeddings_1 = data_point.curr_encoding_1
        curr_embeddings_2 = data_point.curr_encoding_2

        num_prev_tokens = entity_embeddings[0].shape[1]

        curr_embeddings_1_copy = copy.deepcopy(curr_embeddings_1)

        for i in ids_to_move:
            curr_embeddings_1_copy.last_hidden_state = torch.cat((entity_embeddings[0][:,i:i+1,:], 
                                                                  curr_embeddings_1_copy.last_hidden_state), dim=1)

        model_out_1 = model.generate(encoder_outputs=curr_embeddings_1_copy, 
                                   max_new_tokens=10,
                                   return_dict_in_generate=True, 
                                   output_scores=True)

        out_generated_1 = tokenizer.decode(model_out_1.sequences[0], skip_special_tokens=True)

        curr_embeddings_2_copy = copy.deepcopy(curr_embeddings_2)

        for i in ids_to_move:
            curr_embeddings_2_copy.last_hidden_state = torch.cat((entity_embeddings[0][:,i:i+1,:], 
                                                                  curr_embeddings_2_copy.last_hidden_state), dim=1)

        model_out_2 = model.generate(encoder_outputs=curr_embeddings_2_copy, 
                                   max_new_tokens=10,
                                   return_dict_in_generate=True, 
                                   output_scores=True)

        out_generated_2 = tokenizer.decode(model_out_2.sequences[0], skip_special_tokens=True)

        results_case2.append(TwoSentenceMoveResultCase2(data_point, prev_context, answer_1, answer_2, model_out_1, model_out_2, out_generated_1, out_generated_2))

        print("Time taken:", time.time()-start)
    
    return results_case2

In [16]:
def check_results(results):

    correctly_guessed = []
    wrongly_guessed = []
    guessed_nothing = []
    all_generated = []
    all_len = []
    twice = 0

    for result in results:
        data_point = result.data_point
        prev_context = result.prev_context
        answer_1 = result.answer_1
        answer_2 = result.answer_2
        model_out_1 = result.model_out_1
        model_out_2 = result.model_out_2
        out_generated_1 = result.out_generated_1
        out_generated_2 = result.out_generated_2

        all_generated.append(out_generated_1)
        all_generated.append(out_generated_2)

        all_len.append(len(tokenizer.encode(out_generated_1)))
        all_len.append(len(tokenizer.encode(out_generated_2)))

        if (answer_1 in out_generated_1) and (answer_2 in out_generated_1):
            twice += 1
        if (answer_1 in out_generated_2) and (answer_2 in out_generated_2):
            twice += 1

        if (answer_1 in out_generated_1) and (answer_2 not in out_generated_1):
            correctly_guessed.append(1)
        else:
            correctly_guessed.append(0)
        if (answer_2 in out_generated_2) and (answer_1 not in out_generated_2):
            correctly_guessed.append(1)
        else:
            correctly_guessed.append(0)
            
            

        if (answer_2 in out_generated_1) and (answer_1 not in out_generated_1):
            wrongly_guessed.append(1)
        else:
            wrongly_guessed.append(0)
        if (answer_1 in out_generated_2) and (answer_2 not in out_generated_2):
            wrongly_guessed.append(1)
        else:
            wrongly_guessed.append(0)
            
            

        if (answer_2 not in out_generated_1) and (answer_1 not in out_generated_1):
            guessed_nothing.append(1)
            #print()
            #print(out_generated_1, "---", answer_1, answer_2)
        else:
            guessed_nothing.append(0)
        if (answer_1 not in out_generated_2) and (answer_2 not in out_generated_2):
            guessed_nothing.append(1)
            #print()
            #print(out_generated_2, "---", answer_1, answer_2)
        else:
            guessed_nothing.append(0)



    print("Max len:", max(all_len))
    print("Set len_s:", set(all_len))
    print("Both entitites generated in one generation:", twice)
    print("Accuracy:", sum(correctly_guessed)/len(correctly_guessed))
    print("Wrong acuuracy:", sum(wrongly_guessed)/len(wrongly_guessed))
    print("guessed_nothing:", sum(guessed_nothing)/len(guessed_nothing))

# Case 1 Pre-Encodings

In [4]:
data_points_two_sentences_case1 = []

for name_1 in names_1:
    for name_2 in names_2:
        for city_1 in cities_1:
            for city_2 in cities_2:
                
                sample = DataSample_TwoSentences(name_1, name_2, city_1, city_2)
                
                prev_context = sample.prev_context
                
                prev_tokens = tokenizer.encode(prev_context, return_tensors="pt").to(model.encoder.device)
                with torch.no_grad():
                    encoded_prev = model.encoder(prev_tokens, output_special=True, output_hidden_states=True)
                    
                sample.add_prev_encoding(encoded_prev)
        
                data_points_two_sentences_case1.append(sample)
        

In [5]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/TwoSentenceDataEncodings/data_points_two_sentences_case1.txt", "wb") as f:
    pickle.dump(data_points_two_sentences_case1, f)

# Case 2 Pre-Encodings

In [6]:
data_points_two_sentences_case2 = []

for name_1 in names_1:
    for name_2 in names_2:
        for city_1 in cities_1:
            for city_2 in cities_2:
                
                sample = DataSample_TwoSentences(name_1, name_2, city_1, city_2)
                
                prev_context = sample.prev_context 
                current_context_1 = sample.current_context_1
                current_context_2 = sample.current_context_2
                
                prev_tokens = tokenizer.encode(prev_context, return_tensors="pt").to(model.encoder.device)
                with torch.no_grad():
                    encoded_prev = model.encoder(prev_tokens)
                    
                sample.add_prev_encoding(encoded_prev)

                curr_tokens_1 = tokenizer.encode(current_context_1, return_tensors="pt").to(model.encoder.device)
                with torch.no_grad():
                    encoded_curr_1 = model.encoder(curr_tokens_1)
                sample.add_current_encoding_1(encoded_curr_1)
                
                curr_tokens_2 = tokenizer.encode(current_context_2, return_tensors="pt").to(model.encoder.device)
                with torch.no_grad():
                    encoded_curr_2 = model.encoder(curr_tokens_2)
                sample.add_current_encoding_2(encoded_curr_2)
        
                data_points_two_sentences_case2.append(sample)
        

In [7]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/TwoSentenceDataEncodings/data_points_two_sentences_case2.txt", "wb") as f:
    pickle.dump(data_points_two_sentences_case2, f)

# Move: Char+Loc Tokens

## Case 1

In [14]:
entity_inds = list(range(4))
ids_to_move = [ 0, 6, 4, 10 ]
results_case1_charloc = case1_generation(data_points_two_sentences_case1, ids_to_move, entity_inds)

NOW PROCESSING: 0
Time taken: 1.0386078357696533
NOW PROCESSING: 1
Time taken: 0.790971040725708
NOW PROCESSING: 2
Time taken: 0.7905738353729248
NOW PROCESSING: 3
Time taken: 0.7849323749542236
NOW PROCESSING: 4
Time taken: 0.9381880760192871
NOW PROCESSING: 5
Time taken: 0.8937416076660156
NOW PROCESSING: 6
Time taken: 0.7788777351379395
NOW PROCESSING: 7
Time taken: 0.7790963649749756
NOW PROCESSING: 8
Time taken: 0.7776899337768555
NOW PROCESSING: 9
Time taken: 0.7777361869812012
NOW PROCESSING: 10
Time taken: 0.9072475433349609
NOW PROCESSING: 11
Time taken: 1.0307414531707764
NOW PROCESSING: 12
Time taken: 1.1599161624908447
NOW PROCESSING: 13
Time taken: 0.9051997661590576
NOW PROCESSING: 14
Time taken: 1.0299947261810303
NOW PROCESSING: 15
Time taken: 0.9062380790710449
NOW PROCESSING: 16
Time taken: 1.2887284755706787
NOW PROCESSING: 17
Time taken: 1.2849514484405518
NOW PROCESSING: 18
Time taken: 0.7794015407562256
NOW PROCESSING: 19
Time taken: 1.285465955734253
NOW PROCESSI

In [15]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case1_charloc.txt", "wb") as f:
    pickle.dump(results_case1_charloc, f)

In [51]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case1_charloc.txt", "rb") as f:
    results_case1_charloc = pickle.load(f)

In [52]:
len(results_case1_charloc)

625

In [53]:
check_results(results_case1_charloc)

Max len: 5
Set len_s: {2, 3, 4, 5}
Both entitites generated in one generation: 0
Accuracy: 0.3592
Wrong acuuracy: 0.3152
guessed_nothing: 0.3256


In [14]:
for result in results_case1[93:]:
    data_point = result.data_point
    prev_context = result.prev_context
    answer_1 = result.answer_1
    answer_2 = result.answer_2
    model_out_1 = result.model_out_1
    model_out_2 = result.model_out_2
    out_generated_1 = result.out_generated_1
    out_generated_2 = result.out_generated_2
    
    print(prev_context)
    print(answer_1)
    print(answer_2)
    print(model_out_1)
    print(model_out_2)
    print(out_generated_1)
    print(out_generated_2)
    break
    

John travelled to Istanbul. Olivia travelled to Rome.
Istanbul
Rome
GreedySearchEncoderDecoderOutput(sequences=tensor([[   0,   16,    3,    9, 1207,    1]], device='cuda:0'), scores=(tensor([[-33.1250,  -4.8516,  -9.9141,  ..., -31.7031, -32.7812, -32.4688]],
       device='cuda:0', dtype=torch.float16), tensor([[-35.2188,  -4.8789,  -8.4531,  ..., -34.9062, -34.8438, -34.8438]],
       device='cuda:0', dtype=torch.float16), tensor([[-50.0938, -11.5234,  -7.1875,  ..., -49.8125, -49.6875, -49.9688]],
       device='cuda:0', dtype=torch.float16), tensor([[-32.1875,  -4.9023,  -7.8320,  ..., -31.9375, -31.9688, -31.7656]],
       device='cuda:0', dtype=torch.float16), tensor([[-43.7812,   2.6133,  -9.3203,  ..., -42.7500, -43.3438, -42.9688]],
       device='cuda:0', dtype=torch.float16)), encoder_attentions=None, encoder_hidden_states=None, decoder_attentions=None, cross_attentions=None, decoder_hidden_states=None)
GreedySearchEncoderDecoderOutput(sequences=tensor([[   0,  368, 1060,  

In [18]:
tokenizer.decode([   0,   16,    3,    9, 1207,    1])

'<pad> in a bar</s>'

## Case 2

In [23]:
ids_to_move = [ 0, 6, 4, 10 ]

results_case2_charloc = case2_generation(data_points_two_sentences_case2, ids_to_move)
        

NOW PROCESSING: 0
Time taken: 0.7666771411895752
NOW PROCESSING: 1
Time taken: 0.5904951095581055
NOW PROCESSING: 2
Time taken: 1.0454325675964355
NOW PROCESSING: 3
Time taken: 0.5393891334533691
NOW PROCESSING: 4
Time taken: 0.5415420532226562
NOW PROCESSING: 5
Time taken: 0.6860911846160889
NOW PROCESSING: 6
Time taken: 0.665194034576416
NOW PROCESSING: 7
Time taken: 0.5474185943603516
NOW PROCESSING: 8
Time taken: 0.5429384708404541
NOW PROCESSING: 9
Time taken: 0.5468642711639404
NOW PROCESSING: 10
Time taken: 0.8027060031890869
NOW PROCESSING: 11
Time taken: 0.9358367919921875
NOW PROCESSING: 12
Time taken: 1.0533788204193115
NOW PROCESSING: 13
Time taken: 0.8017306327819824
NOW PROCESSING: 14
Time taken: 0.9297294616699219
NOW PROCESSING: 15
Time taken: 0.8010303974151611
NOW PROCESSING: 16
Time taken: 1.0527963638305664
NOW PROCESSING: 17
Time taken: 1.0527305603027344
NOW PROCESSING: 18
Time taken: 0.6705031394958496
NOW PROCESSING: 19
Time taken: 1.0558209419250488
NOW PROCESS

In [24]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case2_charloc.txt", "wb") as f:
    pickle.dump(results_case2_charloc, f)

In [54]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case2_charloc.txt", "rb") as f:
    results_case2_charloc = pickle.load(f)

In [25]:
len(results_case2_charloc)

625

In [55]:
check_results(results_case2_charloc)

Max len: 6
Set len_s: {2, 3, 4, 5, 6}
Both entitites generated in one generation: 0
Accuracy: 0.2784
Wrong acuuracy: 0.228
guessed_nothing: 0.4936


In [26]:
check_results(results_case2_charloc)

Max len: 6
Set len_s: {2, 3, 4, 5, 6}
Both entitites generated in one generation: 0
Accuracy: 0.2784
Wrong acuuracy: 0.228
guessed_nothing: 0.4936


In [ ]:
next_token_scores = torch.nn.functional.softmax(out_generated_1.scores[0], dim=-1)  # (batch_size * num_beams, vocab_size)

probability = next_token_scores[0][tokenizer.encode(answer)[0]].item()

scores = []
for i, score in enumerate(next_token_scores[0]):
    scores.append( (i, score.item()) )

scores.sort(key=lambda x: x[1], reverse=True)
scores = [(tokenizer.decode(a), b) for a, b in scores]

In [10]:
sum(correctly_guessed[i])/len(correctly_guessed[i])

0.224

In [19]:
sum(correctly_guessed)/len(correctly_guessed)

0.224

In [21]:
for result in results:
    prev_context = result.prev_context
    answer_1 = result.answer_1
    answer_2 = result.answer_2
    model_out_1 = result.model_out_1
    model_out_2 = result.model_out_2
    out_generated_1 = result.out_generated_1
    out_generated_2 = result.out_generated_2
    
    print(prev_context)
    print(out_generated_1)
    print(out_generated_2)
    print()

John travelled to London. Henry travelled to Sydney.
in Australia
in Australia

John travelled to London. Henry travelled to Cairo.
London
London

John travelled to London. Henry travelled to Seoul.
in the hospital
in the hospital

John travelled to London. Henry travelled to Rome.
London
London

John travelled to London. Henry travelled to Prague.
London
in the hospital

John travelled to Paris. Henry travelled to Sydney.
Paris
Paris

John travelled to Paris. Henry travelled to Cairo.
Paris
Paris

John travelled to Paris. Henry travelled to Seoul.
Paris
Paris

John travelled to Paris. Henry travelled to Rome.
Paris
Paris

John travelled to Paris. Henry travelled to Prague.
Paris
Paris

John travelled to Oslo. Henry travelled to Sydney.
in Australia
in Australia

John travelled to Oslo. Henry travelled to Cairo.
in the hospital
in the hospital

John travelled to Oslo. Henry travelled to Seoul.
in the hospital
in the hospital

John travelled to Oslo. Henry travelled to Rome.
in Rome
in 

# Move: Loc Tokens

## Case 1

In [27]:
entity_inds = list(range(2))

ids_to_move = [ 4, 10 ]

results_case1_loctokens = case1_generation(data_points_two_sentences_case1, ids_to_move, entity_inds)
        

NOW PROCESSING: 0
Time taken: 1.1079792976379395
NOW PROCESSING: 1
Time taken: 0.8208644390106201
NOW PROCESSING: 2
Time taken: 0.764298677444458
NOW PROCESSING: 3
Time taken: 0.7680938243865967
NOW PROCESSING: 4
Time taken: 1.0187036991119385
NOW PROCESSING: 5
Time taken: 0.8908381462097168
NOW PROCESSING: 6
Time taken: 0.768104076385498
NOW PROCESSING: 7
Time taken: 0.7670319080352783
NOW PROCESSING: 8
Time taken: 0.8560988903045654
NOW PROCESSING: 9
Time taken: 0.8476040363311768
NOW PROCESSING: 10
Time taken: 1.1465163230895996
NOW PROCESSING: 11
Time taken: 1.1797170639038086
NOW PROCESSING: 12
Time taken: 1.4712207317352295
NOW PROCESSING: 13
Time taken: 0.7742242813110352
NOW PROCESSING: 14
Time taken: 1.017897367477417
NOW PROCESSING: 15
Time taken: 0.8895659446716309
NOW PROCESSING: 16
Time taken: 0.7661049365997314
NOW PROCESSING: 17
Time taken: 1.3922338485717773
NOW PROCESSING: 18
Time taken: 0.7632756233215332
NOW PROCESSING: 19
Time taken: 1.259831190109253
NOW PROCESSING

In [28]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case1_loctokens.txt", "wb") as f:
    pickle.dump(results_case1_loctokens, f)

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case1_loctokens.txt", "rb") as f:
    results_case1_loctokens = pickle.load(f)

In [29]:
check_results(results_case1_loctokens)

Max len: 5
Set len_s: {2, 3, 4, 5}
Both entitites generated in one generation: 0
Accuracy: 0.3576
Wrong acuuracy: 0.312
guessed_nothing: 0.3304


Max len: 5
Set len_s: {2, 3, 4, 5}
Both entitites generated in one generation: 0
Accuracy: 0.3576
Wrong acuuracy: 0.312
guessed_nothing: 0.3304


## Case 2

In [30]:
ids_to_move = [ 4, 10 ]

results_case2_loctokens = case2_generation(data_points_two_sentences_case2, ids_to_move)
        

NOW PROCESSING: 0
Time taken: 0.5286850929260254
NOW PROCESSING: 1
Time taken: 0.521662712097168
NOW PROCESSING: 2
Time taken: 0.777411937713623
NOW PROCESSING: 3
Time taken: 0.5291328430175781
NOW PROCESSING: 4
Time taken: 0.5270652770996094
NOW PROCESSING: 5
Time taken: 0.529238224029541
NOW PROCESSING: 6
Time taken: 0.526477575302124
NOW PROCESSING: 7
Time taken: 0.5275909900665283
NOW PROCESSING: 8
Time taken: 0.5266590118408203
NOW PROCESSING: 9
Time taken: 0.5287337303161621
NOW PROCESSING: 10
Time taken: 0.651787519454956
NOW PROCESSING: 11
Time taken: 0.8977701663970947
NOW PROCESSING: 12
Time taken: 1.016554355621338
NOW PROCESSING: 13
Time taken: 0.650958776473999
NOW PROCESSING: 14
Time taken: 0.895275354385376
NOW PROCESSING: 15
Time taken: 0.6497328281402588
NOW PROCESSING: 16
Time taken: 0.896364688873291
NOW PROCESSING: 17
Time taken: 1.1418874263763428
NOW PROCESSING: 18
Time taken: 0.6502597332000732
NOW PROCESSING: 19
Time taken: 1.0206513404846191
NOW PROCESSING: 20


In [31]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case2_loctokens.txt", "wb") as f:
    pickle.dump(results_case2_loctokens, f)

In [36]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case2_loctokens.txt", "rb") as f:
    results_case2_loctokens = pickle.load(f)

In [32]:
check_results(results_case2_loctokens)

Max len: 5
Set len_s: {2, 3, 4, 5}
Both entitites generated in one generation: 0
Accuracy: 0.3168
Wrong acuuracy: 0.2752
guessed_nothing: 0.408


Max len: 5
Set len_s: {2, 3, 4, 5}
Both entitites generated in one generation: 0
Accuracy: 0.0032
Wrong acuuracy: 0.0
guessed_nothing: 0.9968


# Move: Char Tokens

## Case 1

In [33]:
entity_inds = list(range(2))

ids_to_move = [ 0, 6 ]

results_case1_chartokens = case1_generation(data_points_two_sentences_case1, ids_to_move, entity_inds)
        

NOW PROCESSING: 0
Time taken: 1.4940063953399658
NOW PROCESSING: 1
Time taken: 1.2567477226257324
NOW PROCESSING: 2
Time taken: 1.2561438083648682
NOW PROCESSING: 3
Time taken: 1.2554066181182861
NOW PROCESSING: 4
Time taken: 1.2505970001220703
NOW PROCESSING: 5
Time taken: 1.2520570755004883
NOW PROCESSING: 6
Time taken: 1.2545826435089111
NOW PROCESSING: 7
Time taken: 1.255664348602295
NOW PROCESSING: 8
Time taken: 1.256037950515747
NOW PROCESSING: 9
Time taken: 1.2573516368865967
NOW PROCESSING: 10
Time taken: 1.2561297416687012
NOW PROCESSING: 11
Time taken: 1.2582530975341797
NOW PROCESSING: 12
Time taken: 1.2580153942108154
NOW PROCESSING: 13
Time taken: 1.2561712265014648
NOW PROCESSING: 14
Time taken: 1.4467213153839111
NOW PROCESSING: 15
Time taken: 1.256657361984253
NOW PROCESSING: 16
Time taken: 1.2574775218963623
NOW PROCESSING: 17
Time taken: 1.4654600620269775
NOW PROCESSING: 18
Time taken: 1.3188772201538086
NOW PROCESSING: 19
Time taken: 1.2721526622772217
NOW PROCESSIN

In [34]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case1_chartokens.txt", "wb") as f:
    pickle.dump(results_case1_chartokens, f)

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case1_chartokens.txt", "rb") as f:
    results_case1_chartokens = pickle.load(f)

In [35]:
check_results(results_case1_chartokens)

Max len: 9
Set len_s: {2, 3, 4, 5, 7, 9}
Both entitites generated in one generation: 0
Accuracy: 0.0144
Wrong acuuracy: 0.0
guessed_nothing: 0.9856


Max len: 9
Set len_s: {2, 3, 4, 5, 6, 7, 8, 9}
Both entitites generated in one generation: 0
Accuracy: 0.0136
Wrong acuuracy: 0.0
guessed_nothing: 0.9864


## Case 2

In [36]:
ids_to_move = [ 0, 6 ]

results_case2_chartokens = case2_generation(data_points_two_sentences_case2, ids_to_move)

NOW PROCESSING: 0
Time taken: 1.0221941471099854
NOW PROCESSING: 1
Time taken: 1.0180087089538574
NOW PROCESSING: 2
Time taken: 1.0234906673431396
NOW PROCESSING: 3
Time taken: 1.0197234153747559
NOW PROCESSING: 4
Time taken: 1.0163743495941162
NOW PROCESSING: 5
Time taken: 1.0127801895141602
NOW PROCESSING: 6
Time taken: 1.0152606964111328
NOW PROCESSING: 7
Time taken: 1.0193471908569336
NOW PROCESSING: 8
Time taken: 1.0212876796722412
NOW PROCESSING: 9
Time taken: 1.0197491645812988
NOW PROCESSING: 10
Time taken: 1.0215580463409424
NOW PROCESSING: 11
Time taken: 1.1511430740356445
NOW PROCESSING: 12
Time taken: 1.1778438091278076
NOW PROCESSING: 13
Time taken: 1.0255298614501953
NOW PROCESSING: 14
Time taken: 1.0251460075378418
NOW PROCESSING: 15
Time taken: 1.0228931903839111
NOW PROCESSING: 16
Time taken: 1.0270278453826904
NOW PROCESSING: 17
Time taken: 1.027456521987915
NOW PROCESSING: 18
Time taken: 1.0315349102020264
NOW PROCESSING: 19
Time taken: 1.0292026996612549
NOW PROCESS

In [37]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case2_chartokens.txt", "wb") as f:
    pickle.dump(results_case2_chartokens, f)

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case2_chartokens.txt", "rb") as f:
    results_case2_chartokens = pickle.load(f)

In [38]:
check_results(results_case2_chartokens)

Max len: 5
Set len_s: {2, 3, 4, 5}
Both entitites generated in one generation: 0
Accuracy: 0.0032
Wrong acuuracy: 0.0
guessed_nothing: 0.9968


Max len: 5
Set len_s: {2, 3, 4, 5}
Both entitites generated in one generation: 0
Accuracy: 0.0032
Wrong acuuracy: 0.0
guessed_nothing: 0.9968


# Move: Loc + to Tokens

## Case 1

In [39]:
entity_inds = list(range(4))

ids_to_move = [ 3, 4, 9, 10 ]

results_case1_locandto = case1_generation(data_points_two_sentences_case1, ids_to_move, entity_inds)
        

NOW PROCESSING: 0
Time taken: 1.01578688621521
NOW PROCESSING: 1
Time taken: 0.7743914127349854
NOW PROCESSING: 2
Time taken: 0.9643476009368896
NOW PROCESSING: 3
Time taken: 0.832371711730957
NOW PROCESSING: 4
Time taken: 0.8582422733306885
NOW PROCESSING: 5
Time taken: 0.7749111652374268
NOW PROCESSING: 6
Time taken: 0.7720422744750977
NOW PROCESSING: 7
Time taken: 0.7704510688781738
NOW PROCESSING: 8
Time taken: 0.773188591003418
NOW PROCESSING: 9
Time taken: 0.7721116542816162
NOW PROCESSING: 10
Time taken: 0.768730640411377
NOW PROCESSING: 11
Time taken: 1.018885850906372
NOW PROCESSING: 12
Time taken: 1.0229718685150146
NOW PROCESSING: 13
Time taken: 0.7734391689300537
NOW PROCESSING: 14
Time taken: 1.021226406097412
NOW PROCESSING: 15
Time taken: 0.8898510932922363
NOW PROCESSING: 16
Time taken: 0.7698767185211182
NOW PROCESSING: 17
Time taken: 1.2653779983520508
NOW PROCESSING: 18
Time taken: 0.7671706676483154
NOW PROCESSING: 19
Time taken: 0.7678577899932861
NOW PROCESSING: 2

In [40]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case1_locandto.txt", "wb") as f:
    pickle.dump(results_case1_locandto, f)

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case1_locandto.txt", "rb") as f:
    results_case1_locandto = pickle.load(f)

In [41]:
check_results(results_case1_locandto)

Max len: 5
Set len_s: {2, 3, 4, 5}
Both entitites generated in one generation: 0
Accuracy: 0.3896
Wrong acuuracy: 0.3848
guessed_nothing: 0.2256


## Case 2

In [42]:
ids_to_move = [ 3, 4, 9, 10 ]

results_case2_locandto = case2_generation(data_points_two_sentences_case2, ids_to_move)

NOW PROCESSING: 0
Time taken: 0.6713025569915771
NOW PROCESSING: 1
Time taken: 0.5554466247558594
NOW PROCESSING: 2
Time taken: 0.5345456600189209
NOW PROCESSING: 3
Time taken: 0.5314228534698486
NOW PROCESSING: 4
Time taken: 0.5302367210388184
NOW PROCESSING: 5
Time taken: 0.5289876461029053
NOW PROCESSING: 6
Time taken: 0.5337998867034912
NOW PROCESSING: 7
Time taken: 0.5300993919372559
NOW PROCESSING: 8
Time taken: 0.5331134796142578
NOW PROCESSING: 9
Time taken: 0.5325539112091064
NOW PROCESSING: 10
Time taken: 0.5311837196350098
NOW PROCESSING: 11
Time taken: 0.7877156734466553
NOW PROCESSING: 12
Time taken: 0.9042840003967285
NOW PROCESSING: 13
Time taken: 0.6564486026763916
NOW PROCESSING: 14
Time taken: 0.7832515239715576
NOW PROCESSING: 15
Time taken: 0.658837080001831
NOW PROCESSING: 16
Time taken: 0.5339915752410889
NOW PROCESSING: 17
Time taken: 0.7786457538604736
NOW PROCESSING: 18
Time taken: 0.5322003364562988
NOW PROCESSING: 19
Time taken: 0.7821168899536133
NOW PROCESS

In [43]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case2_locandto.txt", "wb") as f:
    pickle.dump(results_case2_locandto, f)

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case2_locandto.txt", "rb") as f:
    results_case2_locandto = pickle.load(f)

In [44]:
check_results(results_case2_locandto)

Max len: 6
Set len_s: {2, 3, 4, 5, 6}
Both entitites generated in one generation: 0
Accuracy: 0.3496
Wrong acuuracy: 0.3016
guessed_nothing: 0.3488


# Move: Char + Loc + to Tokens

## Case 1

In [45]:
entity_inds = list(range(6))

ids_to_move = [ 0, 6, 3, 4, 9, 10 ]

results_case1_charandlocandto = case1_generation(data_points_two_sentences_case1, ids_to_move, entity_inds)
        

NOW PROCESSING: 0
Time taken: 1.0028183460235596
NOW PROCESSING: 1
Time taken: 0.7680325508117676
NOW PROCESSING: 2
Time taken: 0.7672607898712158
NOW PROCESSING: 3
Time taken: 0.7634446620941162
NOW PROCESSING: 4
Time taken: 0.7655854225158691
NOW PROCESSING: 5
Time taken: 0.9316999912261963
NOW PROCESSING: 6
Time taken: 0.7894487380981445
NOW PROCESSING: 7
Time taken: 0.7666802406311035
NOW PROCESSING: 8
Time taken: 0.7654995918273926
NOW PROCESSING: 9
Time taken: 0.7705075740814209
NOW PROCESSING: 10
Time taken: 0.7644097805023193
NOW PROCESSING: 11
Time taken: 1.0107052326202393
NOW PROCESSING: 12
Time taken: 1.0136642456054688
NOW PROCESSING: 13
Time taken: 0.7698237895965576
NOW PROCESSING: 14
Time taken: 1.015934705734253
NOW PROCESSING: 15
Time taken: 0.7656476497650146
NOW PROCESSING: 16
Time taken: 0.7694668769836426
NOW PROCESSING: 17
Time taken: 1.2466886043548584
NOW PROCESSING: 18
Time taken: 0.8041152954101562
NOW PROCESSING: 19
Time taken: 0.781304121017456
NOW PROCESSI

In [46]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case1_charandlocandto.txt", "wb") as f:
    pickle.dump(results_case1_charandlocandto, f)

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case1_charandlocandto.txt", "rb") as f:
    results_case1_charandlocandto = pickle.load(f)

In [47]:
check_results(results_case1_charandlocandto)

Max len: 5
Set len_s: {2, 3, 4, 5}
Both entitites generated in one generation: 0
Accuracy: 0.3984
Wrong acuuracy: 0.3704
guessed_nothing: 0.2312


## Case 2

In [48]:
ids_to_move = [ 0, 6, 3, 4, 9, 10 ]

results_case2_charandlocandto = case2_generation(data_points_two_sentences_case2, ids_to_move)

NOW PROCESSING: 0
Time taken: 0.7402834892272949
NOW PROCESSING: 1
Time taken: 0.5796468257904053
NOW PROCESSING: 2
Time taken: 0.5293619632720947
NOW PROCESSING: 3
Time taken: 0.530226469039917
NOW PROCESSING: 4
Time taken: 0.5273184776306152
NOW PROCESSING: 5
Time taken: 0.5312612056732178
NOW PROCESSING: 6
Time taken: 0.53684401512146
NOW PROCESSING: 7
Time taken: 0.5325114727020264
NOW PROCESSING: 8
Time taken: 0.531956672668457
NOW PROCESSING: 9
Time taken: 0.5305826663970947
NOW PROCESSING: 10
Time taken: 0.653515100479126
NOW PROCESSING: 11
Time taken: 1.0233325958251953
NOW PROCESSING: 12
Time taken: 1.0209271907806396
NOW PROCESSING: 13
Time taken: 0.6546249389648438
NOW PROCESSING: 14
Time taken: 0.7817175388336182
NOW PROCESSING: 15
Time taken: 0.651416540145874
NOW PROCESSING: 16
Time taken: 1.279052972793579
NOW PROCESSING: 17
Time taken: 1.028620958328247
NOW PROCESSING: 18
Time taken: 0.5293576717376709
NOW PROCESSING: 19
Time taken: 1.0332953929901123
NOW PROCESSING: 20

In [49]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case2_charandlocandto.txt", "wb") as f:
    pickle.dump(results_case2_charandlocandto, f)

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Path Creation/Results/TwoSentence/results_case2_charandlocandto.txt", "rb") as f:
    results_case2_charandlocandto = pickle.load(f)

In [50]:
check_results(results_case2_charandlocandto)

Max len: 6
Set len_s: {2, 3, 4, 5, 6}
Both entitites generated in one generation: 0
Accuracy: 0.3208
Wrong acuuracy: 0.2688
guessed_nothing: 0.4104


In [ ]:
template = tm11
context_current = "Lucas was 30 years old."

context_previous = "John travelled to Oslo. Emma travelled to Sydney."
char_1 = "John"
char_2 = "Emma"
answer_1 = "Oslo"
answer_2 = "Sydney"

question_1 = "Where did John travel to?"
question_2 = "Where did Emma travel to?"

entities_list = ["John", " ", "travelled", "to", "Oslo", ".", "Emma", " ", "travelled", "to", "Sydney", "."]

entity_hidden_states = previous_timestep(context_previous,  entities_list)

to_prepend = " ".join(entities_list)

prompt_current = to_prepend + " " + context_current

probability_correct_1, probability_wrong_1, _ = current_timestep_regular(prompt_current, question_1, answer_1, answer_2, template)
probability_correct_2, probability_wrong_2, _ = current_timestep_regular(prompt_current, question_2, answer_2, answer_1, template)

probability_correct_enhanced_1, probability_wrong_enhanced_1, _ = current_timestep_enhanced(prompt_current, 
                                                                      question_1, 
                                                                      answer_1, 
                                                                      answer_2,
                                                                      template, 
                                                                      entities_list, 
                                                                      entity_hidden_states)

probability_correct_enhanced_2, probability_wrong_enhanced_2, _ = current_timestep_enhanced(prompt_current, 
                                                                      question_2, 
                                                                      answer_2,
                                                                      answer_1,
                                                                      template, 
                                                                      entities_list, 
                                                                      entity_hidden_states)

improvement_correct_1 = probability_correct_enhanced_1 - probability_correct_1
improvement_correct_2 = probability_correct_enhanced_2 - probability_correct_2
improvement_wrong_1 = probability_wrong_enhanced_1 - probability_wrong_1
improvement_wrong_2 = probability_wrong_enhanced_2 - probability_wrong_2
improvements_correct_1.append( improvement_correct_1 )
improvements_correct_2.append( improvement_correct_2 )
improvements_wrong_1.append( improvement_wrong_1 )
improvements_wrong_2.append( improvement_wrong_2 )

#NOT RUN

if probability_correct_1 > probability_wrong_1:
    correct_guesses.append(1)
else:
    correct_guesses.append(0)
if probability_correct_enhanced_1 > probability_wrong_enhanced_1:
    correct_guesses.append(1)
else:
    correct_guesses.append(0) 

# Open-Ended Generation - Two chars two locs

In [3]:
def find_index_one(input_ids, entity_str, index):
    
    entity_id = tokenizer.encode(entity_str)
    
    if len(entity_id) != 2:
        print("Not an appropriate entity!")
        return
    
    entity_id = entity_id[0]
    
    input_ids_list = input_ids.tolist()

    all_entity_mention_indices = []
    for i, j in enumerate(input_ids_list[0]):
        if j == entity_id:
            all_entity_mention_indices.append(i)
    try:
        entity_ind = all_entity_mention_indices[index]
        return entity_ind
    except:
        print("entity not found in the input!")
        return

In [4]:
def previous_timestep(context, entities):
    
    input_ids = tokenizer.encode(context, return_tensors="pt")
    len_input_ids = len(input_ids[0])
    out = model.encoder(input_ids, output_special=True, output_hidden_states=True)
    special_hidden = out.special_hidden_states # 24 x (1, T, d)

    special_reformatted = torch.zeros(num_layers, len_input_ids, d_model) # (24, T, d)
    for i, hidden in enumerate(special_hidden):
        special_reformatted[i:i+1, :, :] = hidden
    
    entity_ind = find_index_one(input_ids, entities[0], 0)
    entities_hidden_states = special_reformatted[:, entity_ind, :].unsqueeze(0)

    for entity in entities[1:]:
        entity_ind = find_index_one(input_ids, entity, 0)
        entity_hidden_states = special_reformatted[:, entity_ind, :].unsqueeze(0)
        entities_hidden_states = torch.cat((entities_hidden_states,
                                       entity_hidden_states), dim=0)
    
    return entities_hidden_states

def current_timestep_regular(context, question, answer_correct, answer_wrong, template):
    
    prompt = template.render(context=context, question=question)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    out = model.generate(input_ids, max_new_tokens=1, return_dict_in_generate=True, output_scores=True)
    next_token_scores = torch.nn.functional.softmax(
                    out.scores[0].float(), dim=-1
                )  # (batch_size * num_beams, vocab_size)

    probability_correct = next_token_scores[0][tokenizer.encode(answer_correct)[0]].item()
    probability_wrong = next_token_scores[0][tokenizer.encode(answer_wrong)[0]].item()

    scores = []
    for i, score in enumerate(next_token_scores[0]):
        scores.append( (i, score.item()) )

    scores.sort(key=lambda x: x[1], reverse=True)
    scores = [(tokenizer.decode(a), b) for a, b in scores]

    return probability_correct, probability_wrong, scores

def current_timestep_enhanced(context, question, answer_correct, answer_wrong, template, entities, entity_hidden_states):
    
    prompt = template.render(context=context, question=question)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    entity_inds = []
    for entity in entities:
        entity_inds.append(find_index_one(input_ids, entity, 0))
        
    out = model.generate(
        input_ids=input_ids,
        entity_hidden_states=entity_hidden_states,
        entity_inds=entity_inds,
        max_new_tokens=1,
        return_dict_in_generate=True,
        output_scores=True
    ) 
    next_token_scores = torch.nn.functional.softmax(
                    out.scores[0].float(), dim=-1
                )  # (batch_size * num_beams, vocab_size)
    
    probability_correct = next_token_scores[0][tokenizer.encode(answer_correct)[0]].item()
    probability_wrong = next_token_scores[0][tokenizer.encode(answer_wrong)[0]].item()

    scores = []
    for i, score in enumerate(next_token_scores[0]):
        scores.append( (i, score.item()) )

    scores.sort(key=lambda x: x[1], reverse=True)
    scores = [(tokenizer.decode(a), b) for a, b in scores]

    return probability_correct, probability_wrong, scores


In [17]:
len(data_points)

625

In [20]:
template = tm11
context_current = "Lucas was 30 years old."
improvements_correct_1 = []
improvements_correct_2 = []
improvements_wrong_1 = []
improvements_wrong_2 = []
for data_point in data_points:
    
    char_1 = data_point.name_1
    char_2 = data_point.name_2
    answer_1 = data_point.city_1
    answer_2 = data_point.city_2
    context_previous = data_point.context
    
    question_1 = "Where did " + char_1 + " travel to?"
    question_2 = "Where did " + char_2 + " travel to?"
    
    entities_list = [char_1, char_2, answer_1, answer_2]
    
    entity_hidden_states = previous_timestep(context_previous,  entities_list)
    
    to_prepend = " ".join(entities_list)
    
    prompt_current = to_prepend + " " + context_current
    
    probability_correct_1, probability_wrong_1, _ = current_timestep_regular(prompt_current, question_1, answer_1, answer_2, template)
    probability_correct_2, probability_wrong_2, _ = current_timestep_regular(prompt_current, question_2, answer_2, answer_1, template)
    
    probability_correct_enhanced_1, probability_wrong_enhanced_1, _ = current_timestep_enhanced(prompt_current, 
                                                                          question_1, 
                                                                          answer_1, 
                                                                          answer_2,
                                                                          template, 
                                                                          entities_list, 
                                                                          entity_hidden_states)
    
    probability_correct_enhanced_2, probability_wrong_enhanced_2, _ = current_timestep_enhanced(prompt_current, 
                                                                          question_2, 
                                                                          answer_2,
                                                                          answer_1,
                                                                          template, 
                                                                          entities_list, 
                                                                          entity_hidden_states)
    
    improvement_correct_1 = probability_correct_enhanced_1 - probability_correct_1
    improvement_correct_2 = probability_correct_enhanced_2 - probability_correct_2
    improvement_wrong_1 = probability_wrong_enhanced_1 - probability_wrong_1
    improvement_wrong_2 = probability_wrong_enhanced_2 - probability_wrong_2
    improvements_correct_1.append( improvement_correct_1 )
    improvements_correct_2.append( improvement_correct_2 )
    improvements_wrong_1.append( improvement_wrong_1 )
    improvements_wrong_2.append( improvement_wrong_2 )
    
    #NOT RUN
    
    if probability_correct_1 > probability_wrong_1:
        correct_guesses.append(1)
    else:
        correct_guesses.append(0)
    if probability_correct_enhanced_1 > probability_wrong_enhanced_1:
        correct_guesses.append(1)
    else:
        correct_guesses.append(0) 

new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-modeling_t5
new code working-utils
new code working-m

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/predictions/holmes_two_char_two_locs.txt", "wb") as f:
    pickle.dump(results_dict, f)

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/predictions/holmes_two_char_two_locs.txt", "rb") as f:
    results_dict = pickle.load(f)

# Templates

In [5]:
# Multiple Choice

tm1 = Template("""Read the following context and choose the best option to answer the question.
Context: {{ context }}
Question: {{ question }}
Options:
 - {{ answer_choices | join("\n - ") }}""")

tm2 = Template("""{{ context }}
{{ question }}
- {{ answer_choices | join("\n- ") }}""")

tm3 = Template("""{{ context }}
{{ question }}
Pick the correct answer from the following options:
- {{ answer_choices | join("\n- ") }}""")

tm4 = Template("""{{ context }}
According to the above context, choose the best option to answer the following question.
Question: {{ question }}
Options:
- {{answer_choices | join("\n - ")}}
""")

tm5 = Template("""{{ context }}
{{ question }}
Pick the best answer from the following options:
A. {{ answer0 }}
B. {{ answer1 }}
C. {{ answer2 }}
D. {{ answer3 }}""")

tm6 = Template("""{{ context }}
According to the above context, choose the best option to answer the following question.
Question: {{ question }}
Options:
A. {{ answer0 }}
B. {{ answer1 }}
C. {{ answer2 }}
D. {{ answer3 }}""")

tm7 = Template("""{{ context }}
{{ question }}
A. {{ answer0 }}
B. {{ answer1 }}
C. {{ answer2 }}
D. {{ answer3 }}""")

# Open-Ended

tm8 = Template("""Question: "{{question}}"
Context: "{{context}}"
Answer:""")

tm9 = Template("""{{ context }}
Given the paragraph above, please answer correctly the following
question:
{{ question }}""")

tm10 = Template("""Given the following passage
"{{context}}",
answer the following question. Note that the answer is present within
the text.
Question: {{question}}""")

tm11 = Template("{{context}} What is the answer to: {{question}}")

In [15]:
import sys
sys.path.append('/kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/models/')
cp modeling_t5.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/models/t5/
cp utils.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/generation/
cp modeling_outputs.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/

SyntaxError: invalid syntax (676971914.py, line 3)